In [1]:
from ai import ai_match, ai10s, ai11s

ai_match(ai=[ai11s, ai10s])

ai11s VS ai10s
count     win    lose    draw
o        2206       0    7794
x           0    4968    5032
total    2206    4968   12826

ratio     win    lose    draw
o       22.1%    0.0%   77.9%
x        0.0%   49.7%   50.3%
total   11.0%   24.8%   64.1%



In [2]:
from marubatsu import Marubatsu

mb = Marubatsu()
while True:
    winner = mb.play(ai=[ai10s, ai11s])
    if winner == Marubatsu.CIRCLE:
        break

Turn o
...
...
...

Turn x
...
.O.
...

Turn o
...
.o.
..X

Turn x
O..
.o.
..x

Turn o
o.X
.o.
..x

Turn x
o.x
.oO
..x

Turn o
o.x
Xoo
..x

Turn x
o.x
xoo
O.x

Turn o
oXx
xoo
o.x

winner draw
oxx
xoo
oOx

Turn o
...
...
...

Turn x
...
.O.
...

Turn o
..X
.o.
...

Turn x
..x
.o.
O..

Turn o
..x
Xo.
o..

Turn x
..x
xo.
o.O

Turn o
.Xx
xo.
o.o

winner o
.xx
xo.
oOo



In [3]:
mb = Marubatsu()
while True:
    winner = mb.play(ai=[ai10s, ai11s], verbose=False)
    if winner == Marubatsu.CIRCLE:
        break

In [4]:
def restart(self):
    self.initialize_board()
    self.turn = Marubatsu.CIRCLE     
    self.move_count = 0
    self.status = Marubatsu.PLAYING
    self.last_move = -1, -1          
    self.last_turn = None
    self.records = []

Marubatsu.restart = restart

In [5]:
def move(self, x, y):
    if self.place_mark(x, y, self.turn):
        self.last_turn = self.turn
        self.turn = Marubatsu.CROSS if self.turn == Marubatsu.CIRCLE else Marubatsu.CIRCLE  
        self.move_count += 1
        self.status = self.judge()
        self.last_move = x, y
        self.records.append(self.last_move)
            
Marubatsu.move = move

In [6]:
mb = Marubatsu()
while True:
    winner = mb.play(ai=[ai10s, ai11s], verbose=False)
    if winner == Marubatsu.CIRCLE:
        print(mb.records)
        break

[(1, 1), (0, 2), (2, 0), (2, 1), (1, 0), (2, 2), (1, 2)]


In [7]:
mb = Marubatsu()
while True:
    winner = mb.play(ai=[ai10s, ai11s], verbose=False)
    if winner == Marubatsu.CIRCLE:
        records = mb.records
        mb.restart()
        for x, y in records:
            mb.move(x, y)
            print(mb)
        break

Turn x
...
.O.
...

Turn o
X..
.o.
...

Turn x
x..
.o.
..O

Turn o
x..
.o.
.Xo

Turn x
x.O
.o.
.xo

Turn o
x.o
.o.
Xxo

winner o
x.o
.oO
xxo



In [8]:
def show_progress(ai, winner):
    mb = Marubatsu()
    while True:
        winner = mb.play(ai=ai, verbose=False)
        if winner == Marubatsu.CIRCLE:
            records = mb.records
            mb.restart()
            for x, y in records:
                mb.move(x, y)
                print(mb)
            break

In [9]:
show_progress([ai10s, ai11s], Marubatsu.CIRCLE)

Turn x
...
.O.
...

Turn o
X..
.o.
...

Turn x
x..
.o.
..O

Turn o
x..
.o.
.Xo

Turn x
x.O
.o.
.xo

Turn o
x.o
.o.
Xxo

winner o
x.o
.oO
xxo



In [10]:
mb = Marubatsu()
mb.move(1, 1)
ai11s(mb, debug=True)

Start ai_by_score
Turn x
...
.O.
...

legal_moves [(0, 0), (1, 0), (2, 0), (0, 1), (2, 1), (0, 2), (1, 2), (2, 2)]
move (0, 0)
Turn o
X..
.o.
...

score -1 best score -inf
UPDATE
  best score -1
  best moves [(0, 0)]
move (1, 0)
Turn o
.X.
.o.
...

score -2 best score -1
move (2, 0)
Turn o
..X
.o.
...

score -1 best score -1
APPEND
  best moves [(0, 0), (2, 0)]
move (0, 1)
Turn o
...
Xo.
...

score -2 best score -1
move (2, 1)
Turn o
...
.oX
...

score -2 best score -1
move (0, 2)
Turn o
...
.o.
X..

score -1 best score -1
APPEND
  best moves [(0, 0), (2, 0), (0, 2)]
move (1, 2)
Turn o
...
.o.
.X.

score -2 best score -1
move (2, 2)
Turn o
...
.o.
..X

score -1 best score -1
APPEND
  best moves [(0, 0), (2, 0), (0, 2), (2, 2)]
Finished
best score -1
best moves [(0, 0), (2, 0), (0, 2), (2, 2)]


(2, 2)

In [11]:
from pprint import pprint
from ai import ai_by_score
from marubatsu import Markpat

def ai11s(mb, debug=False):
    def eval_func(mb):      
        # 真ん中のマスに着手している場合は、評価値として 300 を返す
        if mb.last_move == (1, 1):
            return 300
    
        # 自分が勝利している場合は、評価値として 200 を返す
        if mb.status == mb.last_turn:
            return 200

        markpats = mb.count_markpats()
        if debug:
            pprint(markpats)
        # 相手が勝利できる場合は評価値として -100 を返す
        if markpats[Markpat(last_turn=0, turn=2, empty=1)] > 0:
            return -100
        # 次の自分の手番で自分が必ず勝利できる場合は評価値として 100 を返す
        elif markpats[Markpat(last_turn=2, turn=0, empty=1)] >= 2:
            return 100

        # 評価値の合計を計算する変数を 0 で初期化する
        score = 0        
        # 次の自分の手番で自分が勝利できる場合は評価値に 1 を加算する
        if markpats[Markpat(last_turn=2, turn=0, empty=1)] == 1:
            score += 1
        # 「自 1 敵 0 空 2」の数だけ、評価値を加算する
        score += markpats[Markpat(last_turn=1, turn=0, empty=2)]
        # 「自 0 敵 1 空 2」の数だけ、評価値を減算する
        score -= markpats[Markpat(last_turn=0, turn=1, empty=2)]
        
        # 計算した評価値を返す
        return score

    return ai_by_score(mb, eval_func, debug=debug)

In [12]:
mb = Marubatsu()
mb.move(1, 1)
ai11s(mb, debug=True)

Start ai_by_score
Turn x
...
.O.
...

legal_moves [(0, 0), (1, 0), (2, 0), (0, 1), (2, 1), (0, 2), (1, 2), (2, 2)]
move (0, 0)
Turn o
X..
.o.
...

defaultdict(<class 'int'>,
            {Markpat(last_turn=0, turn=0, empty=3): 2,
             Markpat(last_turn=0, turn=1, empty=2): 3,
             Markpat(last_turn=1, turn=0, empty=2): 2,
             Markpat(last_turn=1, turn=1, empty=1): 1})
score -1 best score -inf
UPDATE
  best score -1
  best moves [(0, 0)]
move (1, 0)
Turn o
.X.
.o.
...

defaultdict(<class 'int'>,
            {Markpat(last_turn=0, turn=0, empty=3): 3,
             Markpat(last_turn=0, turn=1, empty=2): 3,
             Markpat(last_turn=1, turn=0, empty=2): 1,
             Markpat(last_turn=1, turn=1, empty=1): 1})
score -2 best score -1
move (2, 0)
Turn o
..X
.o.
...

defaultdict(<class 'int'>,
            {Markpat(last_turn=0, turn=0, empty=3): 2,
             Markpat(last_turn=0, turn=1, empty=2): 3,
             Markpat(last_turn=1, turn=0, empty=2): 2,
        

(2, 2)

In [13]:
def ai9s(mb, debug=False):
    def eval_func(mb):
        # 真ん中のマスに着手している場合は、評価値として 4 を返す
        if mb.last_move == (1, 1):
            return 4
    
        # 自分が勝利している場合は、評価値として 3 を返す
        if mb.status == mb.last_turn:
            return 3

        markpats = mb.count_markpats()
        if debug:
            pprint(markpats)
        # 相手が勝利できる場合は評価値として -1 を返す
        if markpats[Markpat(last_turn=0, turn=2, empty=1)] > 0:
            return -1
        # 次の自分の手番で自分が必ず勝利できる場合は評価値として 2 を返す
        elif markpats[Markpat(last_turn=2, turn=0, empty=1)] >= 2:
            return 2
        # 次の自分の手番で自分が勝利できる場合は評価値として 1 を返す
        elif markpats[Markpat(last_turn=2, turn=0, empty=1)] == 1:
            return 1
        # それ以外の場合は評価値として 0 を返す
        else:
            return 0

    return ai_by_score(mb, eval_func, debug=debug) 

In [14]:
def ai10s(mb, debug=False):
    def eval_func(mb):      
        # 真ん中のマスに着手している場合は、評価値として 300 を返す
        if mb.last_move == (1, 1):
            return 300
    
        # 自分が勝利している場合は、評価値として 200 を返す
        if mb.status == mb.last_turn:
            return 200

        markpats = mb.count_markpats()
        if debug:
            pprint(markpats)
        # 相手が勝利できる場合は評価値として -100 を返す
        if markpats[Markpat(last_turn=0, turn=2, empty=1)] > 0:
            return -100
        # 次の自分の手番で自分が必ず勝利できる場合は評価値として 100 を返す
        elif markpats[Markpat(last_turn=2, turn=0, empty=1)] >= 2:
            return 100

        # 評価値の合計を計算する変数を 0 で初期化する
        score = 0        
        # 次の自分の手番で自分が勝利できる場合は評価値に 1 を加算する
        if markpats[Markpat(last_turn=2, turn=0, empty=1)] == 1:
            score += 1
        # 「自 1 敵 0 空 2」の数だけ、評価値を加算する
        score += markpats[Markpat(last_turn=1, turn=0, empty=2)]
        
        # 計算した評価値を返す
        return score

    return ai_by_score(mb, eval_func, debug=debug)

In [15]:
a = 1 # グローバル変数 a に 1 を代入する
b = 2 # グローバル変数 b に 2 を代入する

def x():
    def y():
        global b                 # b をグローバル変数とみなす
        print("global b =", b)   # グローバル変数 b を表示する

    def z():
        nonlocal b               # b を 1 つ外側の名前空間である x のローカル変数とみなす
        print("nonlocal b =", b) # x のローカル変数 b を表示する

    global a                # a をグローバル変数とみなす
    print("global a =", a)  # グローバル変数 a を表示する
    b = 3                   # 関数 x のローカル変数 b に 3 を代入する
    y()                     # ローカル関数 y を呼び出す
    z()                     # ローカル関数 z を呼び出す
      
x() # 関数 x を呼び出す

global a = 1
global b = 2
nonlocal b = 3


In [16]:
mb.move(0, 0)
ai10s(mb, debug=True)

Start ai_by_score
Turn o
X..
.o.
...

legal_moves [(1, 0), (2, 0), (0, 1), (2, 1), (0, 2), (1, 2), (2, 2)]
move (1, 0)
Turn x
xO.
.o.
...

defaultdict(<class 'int'>,
            {Markpat(last_turn=0, turn=0, empty=3): 2,
             Markpat(last_turn=0, turn=1, empty=2): 1,
             Markpat(last_turn=1, turn=0, empty=2): 2,
             Markpat(last_turn=1, turn=1, empty=1): 2,
             Markpat(last_turn=2, turn=0, empty=1): 1})
score 3 best score -inf
UPDATE
  best score 3
  best moves [(1, 0)]
move (2, 0)
Turn x
x.O
.o.
...

defaultdict(<class 'int'>,
            {Markpat(last_turn=0, turn=0, empty=3): 1,
             Markpat(last_turn=0, turn=1, empty=2): 1,
             Markpat(last_turn=1, turn=0, empty=2): 3,
             Markpat(last_turn=1, turn=1, empty=1): 2,
             Markpat(last_turn=2, turn=0, empty=1): 1})
score 4 best score 3
UPDATE
  best score 4
  best moves [(2, 0)]
move (0, 1)
Turn x
x..
Oo.
...

defaultdict(<class 'int'>,
            {Markpat(last_turn=

(2, 2)

In [17]:
mb.move(2, 2)
ai11s(mb, debug=True)

Start ai_by_score
Turn x
x..
.o.
..O

legal_moves [(1, 0), (2, 0), (0, 1), (2, 1), (0, 2), (1, 2)]
move (1, 0)
Turn o
xX.
.o.
..o

defaultdict(<class 'int'>,
            {Markpat(last_turn=0, turn=1, empty=2): 4,
             Markpat(last_turn=1, turn=0, empty=2): 1,
             Markpat(last_turn=1, turn=1, empty=1): 1,
             Markpat(last_turn=1, turn=2, empty=0): 1,
             Markpat(last_turn=2, turn=0, empty=1): 1})
score -2 best score -inf
UPDATE
  best score -2
  best moves [(1, 0)]
move (2, 0)
Turn o
x.X
.o.
..o

defaultdict(<class 'int'>,
            {Markpat(last_turn=0, turn=1, empty=2): 3,
             Markpat(last_turn=1, turn=0, empty=2): 1,
             Markpat(last_turn=1, turn=1, empty=1): 2,
             Markpat(last_turn=1, turn=2, empty=0): 1,
             Markpat(last_turn=2, turn=0, empty=1): 1})
score -1 best score -2
UPDATE
  best score -1
  best moves [(2, 0)]
move (0, 1)
Turn o
x..
Xo.
..o

defaultdict(<class 'int'>,
            {Markpat(last_turn=0, 

(2, 1)

In [18]:
mb.move(1, 2)
ai10s(mb, debug=True)

Start ai_by_score
Turn o
x..
.o.
.Xo

legal_moves [(1, 0), (2, 0), (0, 1), (2, 1), (0, 2)]
move (1, 0)
Turn x
xO.
.o.
.xo

defaultdict(<class 'int'>,
            {Markpat(last_turn=0, turn=1, empty=2): 1,
             Markpat(last_turn=1, turn=0, empty=2): 3,
             Markpat(last_turn=1, turn=1, empty=1): 2,
             Markpat(last_turn=2, turn=1, empty=0): 2})
score 3 best score -inf
UPDATE
  best score 3
  best moves [(1, 0)]
move (2, 0)
Turn x
x.O
.o.
.xo

defaultdict(<class 'int'>,
            {Markpat(last_turn=0, turn=1, empty=2): 1,
             Markpat(last_turn=1, turn=0, empty=2): 1,
             Markpat(last_turn=1, turn=1, empty=1): 3,
             Markpat(last_turn=2, turn=0, empty=1): 2,
             Markpat(last_turn=2, turn=1, empty=0): 1})
score 100 best score 3
UPDATE
  best score 100
  best moves [(2, 0)]
move (0, 1)
Turn x
x..
Oo.
.xo

defaultdict(<class 'int'>,
            {Markpat(last_turn=0, turn=1, empty=2): 1,
             Markpat(last_turn=1, turn=0, e

(2, 0)